In [51]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

In [52]:
#Snippet below runs through each page and gets URL for each listing
pages = np.arange(1, 63, 1)
links = []
for page in pages:
    page = requests.get("https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/58?tp=3000000&pt=5,2,10&page=" + str(page))   
    content = page.text
    soup = BeautifulSoup(content, 'lxml')
    box =soup.findAll('a', class_='listingResult row')
    for page in box:
        links.append(page.get('href'))
        
    

In [53]:
#Create list of full links including root website address
string = 'https://www.privateproperty.co.za'
final_list=[string + s for s in links]
print(final_list)

['https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/heathfield/caversham-villas/5-hawthorne-road/T3402052', 'https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/wynberg/302-1-malton/1-malton-road/T3360249', 'https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/plumstead/32-carnarvan-road/T3331148', 'https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/newlands/613-newlands-peak/6-thicket/T3422104', 'https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/kenilworth/3a-jesmond-road/T3436945', 'https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/tokai/102-royale-on-main/380-main-road/T2390554', 'https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/plumstead/127-stella-road-plumstead/T3462836', 'https://www.privateproperty.co.za/for-sale/western-cape/cape-town/southern-suburbs/plu

In [55]:
titles=[] #Listing name on private property
prices=[] #List to store price of each listing
addresses=[] #List to store price of each listing
features=[] #List of features for each listing

for i in range(0,1479):
    url = final_list[i]
    result = requests.get(url)
    content = result.text  
    soup = BeautifulSoup(content, 'lxml')

    price = soup.find('span', class_='detailsPrice').get_text()
    prices.append(price)

    box =soup.find('div', class_='titleContainer')
    title = box.find('h1').get_text()
    titles.append(title)

    address =soup.find('div', class_='address')
    if address == (None):
        addresses.append('NA')
    else:
        addresses.append(address.text)

    try:
        feature = soup.find('div', class_='mainFeatures').get_text()
        features.append(feature)
    except:
        print("An exception occurred")
        features.append("null")

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred


In [65]:
len(prices)

1479

In [68]:
df = pd.DataFrame({'Listing':titles,'Price':prices, 'Address':addresses, 'Features':features, 'Link':final_list[0:1479]})


In [69]:
temp_df = df["Features"].str.split("\n", n = 4, expand = True)
Floor_space = temp_df[1]
Floor_space = Floor_space.str.replace(r'\D', '')
temp_df[1] = Floor_space

df_final = pd.concat([df, temp_df[1]], axis=1)
df_final = df_final.drop("Features", axis = 'columns')
df_final = df_final.rename (columns = {1: 'Floor_space_m²'})

split = df_final["Listing"].str.split(" ", n = 1, expand = True)
split2 = df_final["Listing"].str.split("in", n = 1, expand = True)
split3 = df_final["Address"].str.split("Address:", n = 10, expand = True)

df_final["Number_bedrooms"] = split[0]
df_final["Area"] = split2[1]
df_final["Address"] = split3[1]
df_final["Number_bedrooms"].replace("Studio", 0.5)
df_final.drop(df_final[df_final['Price'] == "Sold"].index, inplace = True)

df_final['Price'] = df_final['Price'].str.replace(" ","")
df_final['Price'] = df_final['Price'].str.extract('(\d+)', expand=False)

df_final.head()

C:\Users\Jay\AppData\Local\Temp/ipykernel_15396/4143672455.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  Floor_space = Floor_space.str.replace(r'\D', '')


,Listing,Price,Address,Link,Floor_space_m²,Number_bedrooms,Area
0,2 Bed Townhouse in Heathfield,1395000,"caversham villas, 5 hawthorne road",https://www.privateproperty.co.za/for-sale/wes...,91,2,Heathfield
1,2 Bed Apartment in Wynberg Upper,1495000,"302 1 malton, 1 malton road",https://www.privateproperty.co.za/for-sale/wes...,50,2,Wynberg Upper
2,3 Bed House in Plumstead,2200000,32 carnarvan road,https://www.privateproperty.co.za/for-sale/wes...,100,3,Plumstead
3,1 Bed Apartment in Newlands,2225000,"613 newlands peak, 6 thicket",https://www.privateproperty.co.za/for-sale/wes...,49,1,Newlands
4,3 Bed House in Kenilworth,2495000,3a jesmond road,https://www.privateproperty.co.za/for-sale/wes...,132,3,Kenilworth


In [70]:
df_final.to_csv("southern_suburbs_prices.csv")